In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import catboost as cb
import optuna

from sklearn.impute import KNNImputer

In [112]:
df = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
df_res = pd.DataFrame()
df_res['PassengerId'] = df_test['PassengerId']
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [113]:
df.isna().sum(axis=0)

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [114]:
df['PassengerGroup'] = df['PassengerId'].apply( lambda x: x.split('_')[0]).astype(int)
df_test['PassengerGroup'] = df_test['PassengerId'].apply( lambda x: x.split('_')[0]).astype(int)
df['PassengerGroup']

0          1
1          2
2          3
3          3
4          4
        ... 
8688    9276
8689    9278
8690    9279
8691    9280
8692    9280
Name: PassengerGroup, Length: 8693, dtype: int32

In [115]:
def split_cabin(df):
    df['Cabin'].fillna('nann/-1/nann', inplace=True)

    df['Cabin_deck'] = df['Cabin'].apply( lambda x: x.split('/')[0])
    df['Cabin_num'] = df['Cabin'].apply( lambda x: x.split('/')[1]).astype(int)
    df['Cabin_side'] = df['Cabin'].apply( lambda x: x.split('/')[2])

    df['Cabin_deck'] = df['Cabin_deck'].replace('nann', np.nan)
    df['Cabin_num'] = df['Cabin_num'].replace(-1, np.nan)
    df['Cabin_side'] = df['Cabin_side'].replace('nann', np.nan)

    df['Cabin'] = df['Cabin_deck'].replace('nann/-1/nann', np.nan)
    
    return df

In [116]:
df = split_cabin(df)
df_test = split_cabin(df_test)
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,PassengerGroup,Cabin_deck,Cabin_num,Cabin_side
0,0001_01,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,1,B,0.0,P
1,0002_01,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,2,F,0.0,S
2,0003_01,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,3,A,0.0,S
3,0003_02,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,3,A,0.0,S
4,0004_01,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,4,F,1.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,9276,A,98.0,P
8689,9278_01,Earth,True,G,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,9278,G,1499.0,S
8690,9279_01,Earth,False,G,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,9279,G,1500.0,S
8691,9280_01,Europa,False,E,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,9280,E,608.0,S


In [117]:
categorical_features = ['HomePlanet', 'Destination', 'Cabin_deck', 'Cabin_side']
regression_features = ['Age', 'Cabin_num']
bool_features = ['CryoSleep', 'VIP']
spending_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
drop_features = ['PassengerId', 'Cabin', 'Name']

def fillna_drop(df):
    for features in categorical_features:
        df[features].fillna(df[features].mode().iloc[0], inplace=True)
    for features in regression_features:
        df[features].fillna(df[features].median(), inplace=True)
    for features in bool_features:
        df[features] = df[features].astype(float)
    df = df.drop(columns=drop_features)
    
    bool_imputer = KNNImputer(n_neighbors=1)
    spending_imputer = KNNImputer(n_neighbors=4)
    
    df[bool_features] = bool_imputer.fit_transform(df[bool_features])
    df[spending_features] = spending_imputer.fit_transform(df[spending_features])
    
    return df

df = fillna_drop(df)
df_test = fillna_drop(df_test)
df

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,PassengerGroup,Cabin_deck,Cabin_num,Cabin_side
0,Europa,0.0,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1,B,0.0,P
1,Earth,0.0,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,True,2,F,0.0,S
2,Europa,0.0,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,False,3,A,0.0,S
3,Europa,0.0,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,False,3,A,0.0,S
4,Earth,0.0,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,True,4,F,1.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,0.0,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,False,9276,A,98.0,P
8689,Earth,1.0,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,False,9278,G,1499.0,S
8690,Earth,0.0,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,True,9279,G,1500.0,S
8691,Europa,0.0,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,False,9280,E,608.0,S


In [118]:
categorical_features = ['HomePlanet', 'Destination', 'Cabin_deck', 'Cabin_side']

def encode_cat(df):
    for features in categorical_features:
            values = set(df[features])
            for v in values:
                df[features + '_' + v] = df[features] == v
            df = df.drop(columns=features)

    df = df.drop(columns='Cabin_side_S')
    return df

df = encode_cat(df)
df_test = encode_cat(df_test)

In [119]:
target = 'Transported'
y = df[target]
X = df.drop(columns = target).values
X_test = df_test.values

X, y, X_test

(array([[0.0, 39.0, 0.0, ..., False, False, True],
        [0.0, 24.0, 0.0, ..., False, False, False],
        [0.0, 58.0, 1.0, ..., False, False, False],
        ...,
        [0.0, 26.0, 0.0, ..., False, False, False],
        [0.0, 32.0, 0.0, ..., True, False, False],
        [0.0, 44.0, 0.0, ..., True, False, False]], dtype=object),
 0       False
 1        True
 2       False
 3       False
 4        True
         ...  
 8688    False
 8689    False
 8690     True
 8691    False
 8692     True
 Name: Transported, Length: 8693, dtype: bool,
 array([[1.0, 27.0, 0.0, ..., False, False, False],
        [0.0, 19.0, 0.0, ..., False, False, False],
        [1.0, 31.0, 0.0, ..., False, False, False],
        ...,
        [1.0, 26.0, 0.0, ..., False, False, True],
        [0.0, 26.0, 0.0, ..., False, False, True],
        [1.0, 43.0, 0.0, ..., False, False, False]], dtype=object))

In [120]:
def kfold(X, y, k=5):
    pre_mask = np.arange(y.size) % k

    X_trains = []
    y_trains = []
    X_vals = []
    y_vals = []

    for i in range(k):
        val_mask = pre_mask == i
        
        y_vals.append(y[val_mask])
        y_trains.append(y[~val_mask])
        X_vals.append(X[val_mask])
        X_trains.append(X[~val_mask])
    result = []
    for i in range(k):
        result.append(((X_trains[i], y_trains[i]), (X_vals[i], y_vals[i])))
    return result

In [121]:
def optune_optimize(trial):
    param = {
        "depth": trial.suggest_int("depth", 1, 12),
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3)
    }
    model = cb.CatBoostClassifier(**param)
    #model = cb.CatBoostClassifier()
    accs = []
    for ((X_train, y_train), (X_val, y_val)) in kfold(X, y, 10):
        model.fit(X_train, y_train)
        pred = model.predict(X_val)
        pred = pred == 'True'
        accs.append(np.mean(pred == y_val))
    acc_mean = np.mean(accs)
    return acc_mean

In [122]:
study = optuna.create_study(direction="maximize")
study.optimize(optune_optimize, n_trials=10)
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("score на обучающем наборе: {:.2f}".format(study.best_value))

[I 2024-02-22 23:48:52,190] A new study created in memory with name: no-name-68d82c0f-2204-462d-85a5-58309f512a10


0:	learn: 0.5644695	total: 5.45ms	remaining: 1.92s
1:	learn: 0.5159096	total: 11ms	remaining: 1.94s
2:	learn: 0.4663896	total: 16ms	remaining: 1.87s
3:	learn: 0.4465098	total: 21.2ms	remaining: 1.86s
4:	learn: 0.4290187	total: 26.8ms	remaining: 1.87s
5:	learn: 0.4187859	total: 32.1ms	remaining: 1.86s
6:	learn: 0.4101581	total: 37.4ms	remaining: 1.85s
7:	learn: 0.4005704	total: 42.8ms	remaining: 1.85s
8:	learn: 0.3936583	total: 48ms	remaining: 1.84s
9:	learn: 0.3857493	total: 53.3ms	remaining: 1.83s
10:	learn: 0.3806329	total: 58.5ms	remaining: 1.82s
11:	learn: 0.3783452	total: 63.7ms	remaining: 1.81s
12:	learn: 0.3750392	total: 68.6ms	remaining: 1.8s
13:	learn: 0.3708973	total: 73.6ms	remaining: 1.79s
14:	learn: 0.3666478	total: 78.8ms	remaining: 1.78s
15:	learn: 0.3632130	total: 83.8ms	remaining: 1.77s
16:	learn: 0.3594181	total: 88.3ms	remaining: 1.75s
17:	learn: 0.3566452	total: 93.1ms	remaining: 1.74s
18:	learn: 0.3509522	total: 97.9ms	remaining: 1.73s
19:	learn: 0.3469620	total: 1

170:	learn: 0.1469570	total: 827ms	remaining: 885ms
171:	learn: 0.1464982	total: 832ms	remaining: 881ms
172:	learn: 0.1460512	total: 838ms	remaining: 877ms
173:	learn: 0.1455216	total: 843ms	remaining: 872ms
174:	learn: 0.1444737	total: 847ms	remaining: 867ms
175:	learn: 0.1437738	total: 852ms	remaining: 862ms
176:	learn: 0.1430348	total: 857ms	remaining: 857ms
177:	learn: 0.1427031	total: 862ms	remaining: 852ms
178:	learn: 0.1419986	total: 867ms	remaining: 848ms
179:	learn: 0.1416116	total: 872ms	remaining: 843ms
180:	learn: 0.1412117	total: 877ms	remaining: 838ms
181:	learn: 0.1409336	total: 881ms	remaining: 833ms
182:	learn: 0.1407558	total: 886ms	remaining: 828ms
183:	learn: 0.1400732	total: 891ms	remaining: 823ms
184:	learn: 0.1393758	total: 896ms	remaining: 818ms
185:	learn: 0.1390716	total: 900ms	remaining: 813ms
186:	learn: 0.1387602	total: 905ms	remaining: 808ms
187:	learn: 0.1384148	total: 909ms	remaining: 803ms
188:	learn: 0.1378534	total: 914ms	remaining: 798ms
189:	learn: 

343:	learn: 0.0810347	total: 1.66s	remaining: 48.2ms
344:	learn: 0.0808571	total: 1.66s	remaining: 43.4ms
345:	learn: 0.0807519	total: 1.67s	remaining: 38.5ms
346:	learn: 0.0806891	total: 1.67s	remaining: 33.7ms
347:	learn: 0.0804305	total: 1.68s	remaining: 28.9ms
348:	learn: 0.0801853	total: 1.68s	remaining: 24.1ms
349:	learn: 0.0800162	total: 1.69s	remaining: 19.3ms
350:	learn: 0.0798634	total: 1.69s	remaining: 14.5ms
351:	learn: 0.0796357	total: 1.7s	remaining: 9.65ms
352:	learn: 0.0793069	total: 1.7s	remaining: 4.82ms
353:	learn: 0.0790290	total: 1.71s	remaining: 0us
0:	learn: 0.5722730	total: 6.46ms	remaining: 2.28s
1:	learn: 0.5028551	total: 12.9ms	remaining: 2.28s
2:	learn: 0.4631111	total: 19ms	remaining: 2.22s
3:	learn: 0.4363532	total: 25.3ms	remaining: 2.21s
4:	learn: 0.4208274	total: 31.6ms	remaining: 2.21s
5:	learn: 0.4045012	total: 37.8ms	remaining: 2.19s
6:	learn: 0.3972850	total: 43.8ms	remaining: 2.17s
7:	learn: 0.3896383	total: 49.9ms	remaining: 2.16s
8:	learn: 0.3835

172:	learn: 0.1467752	total: 882ms	remaining: 923ms
173:	learn: 0.1459146	total: 887ms	remaining: 918ms
174:	learn: 0.1453513	total: 892ms	remaining: 912ms
175:	learn: 0.1449395	total: 897ms	remaining: 908ms
176:	learn: 0.1446103	total: 904ms	remaining: 904ms
177:	learn: 0.1439129	total: 910ms	remaining: 900ms
178:	learn: 0.1434821	total: 915ms	remaining: 895ms
179:	learn: 0.1430105	total: 921ms	remaining: 890ms
180:	learn: 0.1425581	total: 926ms	remaining: 885ms
181:	learn: 0.1418823	total: 932ms	remaining: 880ms
182:	learn: 0.1413086	total: 936ms	remaining: 875ms
183:	learn: 0.1406029	total: 942ms	remaining: 870ms
184:	learn: 0.1401945	total: 947ms	remaining: 865ms
185:	learn: 0.1396846	total: 951ms	remaining: 859ms
186:	learn: 0.1393495	total: 956ms	remaining: 854ms
187:	learn: 0.1386237	total: 961ms	remaining: 848ms
188:	learn: 0.1383125	total: 966ms	remaining: 843ms
189:	learn: 0.1379352	total: 970ms	remaining: 837ms
190:	learn: 0.1371427	total: 975ms	remaining: 832ms
191:	learn: 

...

276:	learn: 0.4108295	total: 454ms	remaining: 233ms
277:	learn: 0.4108021	total: 455ms	remaining: 231ms
278:	learn: 0.4107735	total: 457ms	remaining: 229ms
279:	learn: 0.4107282	total: 459ms	remaining: 228ms
280:	learn: 0.4107027	total: 460ms	remaining: 226ms
281:	learn: 0.4106757	total: 462ms	remaining: 225ms
282:	learn: 0.4106494	total: 464ms	remaining: 223ms
283:	learn: 0.4106114	total: 466ms	remaining: 221ms
284:	learn: 0.4105764	total: 467ms	remaining: 220ms
285:	learn: 0.4103887	total: 469ms	remaining: 218ms
286:	learn: 0.4103462	total: 471ms	remaining: 217ms
287:	learn: 0.4103156	total: 473ms	remaining: 215ms
288:	learn: 0.4102683	total: 475ms	remaining: 213ms
289:	learn: 0.4102374	total: 476ms	remaining: 212ms
290:	learn: 0.4102071	total: 478ms	remaining: 210ms
291:	learn: 0.4101455	total: 479ms	remaining: 208ms
292:	learn: 0.4101167	total: 481ms	remaining: 207ms
293:	learn: 0.4100774	total: 483ms	remaining: 205ms
294:	learn: 0.4100465	total: 484ms	remaining: 204ms
295:	learn: 

59:	learn: 0.4471630	total: 96.6ms	remaining: 578ms
60:	learn: 0.4465174	total: 98.7ms	remaining: 579ms
61:	learn: 0.4458963	total: 101ms	remaining: 579ms
62:	learn: 0.4453182	total: 102ms	remaining: 577ms
63:	learn: 0.4447938	total: 104ms	remaining: 576ms
64:	learn: 0.4442576	total: 105ms	remaining: 574ms
65:	learn: 0.4437827	total: 107ms	remaining: 573ms
66:	learn: 0.4433076	total: 109ms	remaining: 571ms
67:	learn: 0.4427861	total: 111ms	remaining: 571ms
68:	learn: 0.4422147	total: 112ms	remaining: 569ms
69:	learn: 0.4417348	total: 114ms	remaining: 568ms
70:	learn: 0.4413063	total: 116ms	remaining: 566ms
71:	learn: 0.4407121	total: 117ms	remaining: 565ms
72:	learn: 0.4400516	total: 119ms	remaining: 563ms
73:	learn: 0.4396981	total: 120ms	remaining: 561ms
74:	learn: 0.4391926	total: 122ms	remaining: 559ms
75:	learn: 0.4387996	total: 124ms	remaining: 557ms
76:	learn: 0.4384668	total: 125ms	remaining: 555ms
77:	learn: 0.4379928	total: 127ms	remaining: 554ms
78:	learn: 0.4375434	total: 1

304:	learn: 0.4066693	total: 497ms	remaining: 186ms
305:	learn: 0.4066438	total: 498ms	remaining: 184ms
306:	learn: 0.4066165	total: 500ms	remaining: 182ms
307:	learn: 0.4065872	total: 501ms	remaining: 181ms
308:	learn: 0.4065623	total: 503ms	remaining: 179ms
309:	learn: 0.4065309	total: 504ms	remaining: 177ms
310:	learn: 0.4065121	total: 506ms	remaining: 176ms
311:	learn: 0.4064875	total: 508ms	remaining: 174ms
312:	learn: 0.4064410	total: 509ms	remaining: 172ms
313:	learn: 0.4063511	total: 511ms	remaining: 171ms
314:	learn: 0.4063150	total: 513ms	remaining: 169ms
315:	learn: 0.4062492	total: 514ms	remaining: 168ms
316:	learn: 0.4062301	total: 517ms	remaining: 166ms
317:	learn: 0.4062047	total: 518ms	remaining: 165ms
318:	learn: 0.4061857	total: 520ms	remaining: 163ms
319:	learn: 0.4061579	total: 521ms	remaining: 161ms
320:	learn: 0.4061324	total: 523ms	remaining: 160ms
321:	learn: 0.4061035	total: 525ms	remaining: 158ms
322:	learn: 0.4060690	total: 526ms	remaining: 156ms
323:	learn: 

[I 2024-02-22 23:54:26,693] Trial 9 finished with value: 0.7962746187320608 and parameters: {'depth': 1, 'iterations': 419, 'learning_rate': 0.2641902785714075}. Best is trial 3 with value: 0.8080061637765699.


Наилучшие значения гиперпараметров {'depth': 5, 'iterations': 100, 'learning_rate': 0.07577408019256733}
score на обучающем наборе: 0.81


In [123]:
model = cb.CatBoostClassifier(**study.best_params)
model.fit(X, y)
pred = model.predict(X_test)
pred

0:	learn: 0.6609144	total: 2.99ms	remaining: 296ms
1:	learn: 0.6319631	total: 5.94ms	remaining: 291ms
2:	learn: 0.6079771	total: 8.63ms	remaining: 279ms
3:	learn: 0.5913518	total: 11.3ms	remaining: 271ms
4:	learn: 0.5721850	total: 14ms	remaining: 266ms
5:	learn: 0.5603050	total: 16.6ms	remaining: 260ms
6:	learn: 0.5439167	total: 19.2ms	remaining: 255ms
7:	learn: 0.5344504	total: 21.8ms	remaining: 251ms
8:	learn: 0.5265719	total: 24.7ms	remaining: 249ms
9:	learn: 0.5178609	total: 27.4ms	remaining: 247ms
10:	learn: 0.5107779	total: 30.1ms	remaining: 243ms
11:	learn: 0.5061026	total: 32.9ms	remaining: 241ms
12:	learn: 0.5009310	total: 35.6ms	remaining: 238ms
13:	learn: 0.4936752	total: 38.3ms	remaining: 236ms
14:	learn: 0.4908552	total: 41.1ms	remaining: 233ms
15:	learn: 0.4831738	total: 44ms	remaining: 231ms
16:	learn: 0.4790682	total: 46.9ms	remaining: 229ms
17:	learn: 0.4739209	total: 50.1ms	remaining: 228ms
18:	learn: 0.4703104	total: 53ms	remaining: 226ms
19:	learn: 0.4662885	total: 

array(['True', 'False', 'True', ..., 'True', 'True', 'True'], dtype=object)

In [124]:
df_res['Transported'] = pred
df_res

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [126]:
df_res.to_csv('C:/Users/Юра/Downloads/res1.csv', index=False)